In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [2]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./dataset', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])), batch_size=50, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./dataset', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])), batch_size=50, shuffle=True)

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 10)

    def _softmax(self, x):
        x = torch.exp(x)
        return torch.div(x, torch.sum(x, dim=1, keepdim=True))
        
    def forward(self, x):
        self.x_flatten = x.view(-1, 28*28)
        self.h1 = F.sigmoid(self.fc1(self.x_flatten))
        self.h2 = F.sigmoid(self.fc2(self.h1))
        self.h3 = self.fc3(self.h2)
        self.out = F.log_softmax(self.h3, dim=1)
        return self.out

In [4]:
model = Net()

In [5]:
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [6]:
def train(epoch):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))
    else:
        batch_idx += 1
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data[0]))

In [7]:
def test():
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:
for epoch in range(1, 21):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.355090
Train Epoch: 1 [10000/60000 (17%)]	Loss: 0.504585
Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.293710
Train Epoch: 1 [30000/60000 (50%)]	Loss: 0.496211
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.221962
Train Epoch: 1 [50000/60000 (83%)]	Loss: 0.263295
Train Epoch: 1 [60000/60000 (100%)]	Loss: 0.065452

Test set: Average loss: 0.2259, Accuracy: 9306/10000 (93%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.246153
Train Epoch: 2 [10000/60000 (17%)]	Loss: 0.261098
Train Epoch: 2 [20000/60000 (33%)]	Loss: 0.103261
Train Epoch: 2 [30000/60000 (50%)]	Loss: 0.170720
Train Epoch: 2 [40000/60000 (67%)]	Loss: 0.085460
Train Epoch: 2 [50000/60000 (83%)]	Loss: 0.173876
Train Epoch: 2 [60000/60000 (100%)]	Loss: 0.061193

Test set: Average loss: 0.1935, Accuracy: 9385/10000 (94%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.200934
Train Epoch: 3 [10000/60000 (17%)]	Loss: 0.202646
Train Epoch: 3 [20000/60000 (33%)]	Loss: 0.150363
Train Epoch: 3 [30000/60000 (50%)]	Loss: 